# Weekend Planner with Azure OpenAI Agents Telemetry

Welcome! This notebook demonstrates how to capture GenAI-compliant spans while orchestrating an Azure OpenAI Agents workflow.

## 🎯 What You'll Build

This notebook shows a different scenario (weekend planning) to demonstrate universal OpenTelemetry concepts:
- ✅ Instrumented Azure OpenAI Agents workflow with automatic telemetry
- ✅ Captured GenAI-compliant spans following semantic conventions
- ✅ Exported telemetry to console and optionally Azure Monitor
- ✅ Observed agent orchestration with function tools

## 💡 What You'll Learn

- How to instrument Azure OpenAI Agents with OpenTelemetry
- How automatic instrumentation captures agent lifecycle events
- How to configure telemetry exporters (console, Azure Monitor)
- Best practices for observing multi-agent workflows
- How GenAI semantic conventions apply across different scenarios

> **Note**: This notebook uses a weekend planning scenario instead of Zava retail to show how the same observability patterns apply to any agentic workflow.

Ready to instrument an Azure OpenAI Agents workflow? Let's get started! 🚀

---

## Requirements
Install the supporting packages before running the cells below.

```bash
pip install openai openai-agents rich python-dotenv
pip install opentelemetry-instrumentation-openai-agents-v2
# Optional Azure Monitor exporter
pip install azure-monitor-opentelemetry-exporter
```

Export the following environment variables before executing the notebook (set unused values to an empty string):
- `AZURE_OPENAI_API_KEY`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_VERSION`
- `AZURE_OPENAI_MODEL_NAME`
- `AZURE_AI_FOUNDRY_NAME`
- `AZURE_AOAI_ACCOUNT`
- `AZURE_SUBSCRIPTION_ID`
- `AZURE_RESOURCE_GROUP`
- `AZURE_AISEARCH_ENDPOINT`
- `AZURE_AISEARCH_INDEX`
- `AZURE_AISEARCH_RESOURCE_GROUP`
- `APPLICATION_INSIGHTS_CONNECTION_STRING`

These labs do not assume any other configuration variables are defined.


### Step 1: Import dependencies and configure logging
Load the OpenAI Agents SDK, OpenTelemetry helpers, and supporting utilities. This cell also reads your `.env` file and enables rich logging so you can watch the agent run.


In [ ]:
from __future__ import annotations

import asyncio
import logging
import os
import random
from dataclasses import dataclass
from datetime import datetime
from typing import Callable
from urllib.parse import urlparse

import openai
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled
from dotenv import load_dotenv
from rich.logging import RichHandler

from opentelemetry import trace
from opentelemetry.instrumentation.openai_agents import OpenAIAgentsInstrumentor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter

try:
    from azure.monitor.opentelemetry.exporter import AzureMonitorTraceExporter
except ImportError:  # pragma: no cover - optional dependency
    AzureMonitorTraceExporter = None

load_dotenv(override=True)

logging.basicConfig(level=logging.WARNING, format="%(message)s", datefmt="[%X]", handlers=[RichHandler()])
LOGGER = logging.getLogger("weekend_planner")
LOGGER.setLevel(logging.INFO)

MODEL_NAME = os.environ.get("AZURE_OPENAI_MODEL_NAME") or "gpt-4o-mini"
SERVICE_VERSION = "1.0.0"


### Step 2: Define helpers for capture configuration
These utilities resolve the Azure OpenAI configuration, set the GenAI capture environment variables, and prepare the tracer provider used throughout the notebook.


In [ ]:
@dataclass
class _ApiConfig:
    """Helper describing how to create the Azure OpenAI client."""

    build_client: Callable[[], object]
    model_name: str
    base_url: str
    provider: str


def _set_capture_env(provider: str, base_url: str) -> None:
    """Enable GenAI capture toggles required by the instrumentation layer."""

    capture_defaults = {
        "OTEL_GENAI_CAPTURE_MESSAGES": "true",
        "OTEL_GENAI_CAPTURE_SYSTEM_INSTRUCTIONS": "true",
        "OTEL_GENAI_CAPTURE_TOOL_DEFINITIONS": "true",
        "OTEL_GENAI_EMIT_OPERATION_DETAILS": "true",
        "OTEL_GENAI_PROVIDER_NAME": provider,
    }
    for key, value in capture_defaults.items():
        os.environ.setdefault(key, value)

    parsed = urlparse(base_url)
    if parsed.hostname:
        os.environ.setdefault("OTEL_GENAI_SERVER_ADDRESS", parsed.hostname)
    if parsed.port:
        os.environ.setdefault("OTEL_GENAI_SERVER_PORT", str(parsed.port))


def _resolve_api_config() -> _ApiConfig:
    """Return the client configuration for Azure OpenAI."""

    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"].rstrip("/")
    api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")
    model_name = os.environ.get("AZURE_OPENAI_MODEL_NAME") or "gpt-4o-mini"
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    def _build_client() -> openai.AsyncAzureOpenAI:
        return openai.AsyncAzureOpenAI(
            api_version=api_version,
            azure_endpoint=endpoint,
            api_key=api_key,
        )

    return _ApiConfig(
        build_client=_build_client,
        model_name=model_name,
        base_url=endpoint,
        provider="azure.ai.openai",
    )


def _configure_tracer() -> None:
    """Configure tracer provider and exporter."""

    resource = Resource.create({
        "service.name": "weekend-planner-service",
        "service.namespace": "ignite25",
        "service.version": SERVICE_VERSION,
    })
    provider = TracerProvider(resource=resource)
    connection_string = os.environ.get("APPLICATION_INSIGHTS_CONNECTION_STRING")

    if connection_string and AzureMonitorTraceExporter is not None:
        exporter = AzureMonitorTraceExporter.from_connection_string(connection_string)
        provider.add_span_processor(BatchSpanProcessor(exporter))
        print("[otel] Azure Monitor trace exporter configured")
    else:
        provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))
        if connection_string and AzureMonitorTraceExporter is None:
            print("[otel] Azure Monitor exporter unavailable. Install azure-monitor-opentelemetry-exporter")
        else:
            print("[otel] Console span exporter configured")

    trace.set_tracer_provider(provider)


### Step 3: Initialise OpenTelemetry instrumentation
Run this cell after updating your environment variables to wire up the tracer provider and the OpenAI Agents instrumentor.


In [ ]:
API_CONFIG = _resolve_api_config()
_set_capture_env(API_CONFIG.provider, API_CONFIG.base_url)
_configure_tracer()

OpenAIAgentsInstrumentor().instrument(tracer_provider=trace.get_tracer_provider())
CLIENT = API_CONFIG.build_client()
set_tracing_disabled(False)
print("Instrumentation ready for provider:", API_CONFIG.provider)

### Step 4: Register agent tools and construct the agent
With instrumentation active, declare the reusable tools and build the Weekend Planner agent that the notebook will invoke.


In [ ]:
@function_tool
def get_weather(city: str) -> dict[str, object]:
    LOGGER.info("Getting weather for %s", city)
    if random.random() < 0.05:
        return {"city": city, "temperature": 72, "description": "Sunny"}
    return {"city": city, "temperature": 60, "description": "Rainy"}


@function_tool
def get_activities(city: str, date: str) -> list[dict[str, object]]:
    LOGGER.info("Getting activities for %s on %s", city, date)
    return [
        {"name": "Hiking", "location": city},
        {"name": "Beach", "location": city},
        {"name": "Museum", "location": city},
    ]


@function_tool
def get_current_date() -> str:
    """Gets the current date and returns as YYYY-MM-DD."""

    LOGGER.info("Getting current date")
    return datetime.now().strftime("%Y-%m-%d")


AGENT = Agent(
    name="Weekend Planner",
    instructions=(
        "You help users plan their weekends and choose the best activities for the given weather. "
        "If an activity would be unpleasant in the weather, do not recommend it. "
        "Always include the relevant weekend date in your response."
    ),
    tools=[get_weather, get_activities, get_current_date],
    model=OpenAIChatCompletionsModel(model=API_CONFIG.model_name, openai_client=CLIENT),
)
print("Agent ready:", AGENT.name)

### Step 5: Run the agent
Execute the final cell to orchestrate a single planning session. Watch the console (or Application Insights) for `create_agent`, `invoke_agent`, and `execute_tool` spans emitted by the instrumentation.


In [ ]:
async def run_planner():
    tracer = trace.get_tracer(__name__)
    with tracer.start_as_current_span(f"weekend_planning_session[{API_CONFIG.provider}]") as span:
        user_request = "Hi, what can I do this weekend in Seattle?"
        span.set_attribute("user.request", user_request)
        span.set_attribute("gen_ai.provider.name", API_CONFIG.provider)
        span.set_attribute("gen_ai.request.model", API_CONFIG.model_name)
        span.set_attribute("agent.name", AGENT.name)
        span.set_attribute("target.city", "Seattle")
        try:
            result = await Runner.run(AGENT, input=user_request)
            output = result.final_output or ""
            span.set_attribute("agent.response", output[:500])
            span.set_attribute("request.success", True)
            print(output)
        except Exception as exc:
            span.record_exception(exc)
            span.set_attribute("request.success", False)
            raise

await run_planner()
trace.get_tracer_provider().shutdown()